In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        break
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms,models
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
import cv2
import os
import numpy as np
import pickle as pkl
import torch
from torchvision.utils import save_image
from matplotlib import cm
from PIL import Image
import numpy as ppool

In [ ]:
class LazyLoadDataset(Dataset):
    def __init__(self,path,train=True,transform=None):
        self.notTrain = 0
        self.transform=transform
        path=path+("train/" if train else "test/")
        
        self.pathX=path+"X/"
        if train == False:
            self.pathY = 0
            self.notTrain = 1
        else:
            self.pathY=path+"Y/"
        
        self.data=os.listdir(self.pathX)
        
    def __getitem__(self,idx):
        f=self.data[idx]
        
        #X
        img0=cv2.imread(self.pathX+f+"/rgb/0.png")
        img1=cv2.imread(self.pathX+f+"/rgb/1.png")
        img2=cv2.imread(self.pathX+f+"/rgb/2.png")
        
        depth=np.load(self.pathX+f+"/depth.npy")
        field_id=pkl.load(open(self.pathX+f+"/field_id.pkl","rb"))
        
        if self.notTrain == 1:
            Y = 0
        else:
            Y=np.load(self.pathY+f+".npy")
        
        if self.transform is not None:
            img0=self.transform(img0)
            img1=self.transform(img1)
            img2=self.transform(img2)
        
        norm = ppool.zeros((224,224))
        final1 = cv2.normalize(img0/255,  norm, 0, 1, cv2.NORM_MINMAX)
        final2 = cv2.normalize(img1/255,  norm, 0, 1, cv2.NORM_MINMAX)
        final3 = cv2.normalize(img2/255,  norm, 0, 1, cv2.NORM_MINMAX)
        final4 = cv2.normalize(depth/1000,  norm, 0, 1, cv2.NORM_MINMAX)
        
        if(self.notTrain == 0):
            return (final1,final2,final3,final4,field_id),(Y)
        else:
            return (final1,final2,final3,final4,field_id)
    def __len__(self):
        return len(self.data)
    
    
  

In [ ]:
def transform1(X):
    '''
    Function for augmenting images. 
    It takes original image and corresponding mask and performs the
    same flipping and rotation transforamtions
    '''
    x = X
    
    im = Image.fromarray(np.uint8(x)).convert('RGB')
    
    t1 = transforms.ColorJitter(brightness=0.5, contrast=0.4, saturation=0.4, hue=0.2)
    im = t1(im)
    
    x = np.array(im)
    # axes swap:
    if np.random.rand()<0.5: # 0.5 chances for this transformation to occur (same for two below)
        x = np.swapaxes(x, 0,1)
#         y = np.swapaxes(y, 0,1)
# vertical flip:
    if np.random.rand()<0.5:
        x = np.flip(x, 0)
#         y = np.flip(y, 0)
# horizontal flip:
    if np.random.rand()<0.5:
        x = np.flip(x, 1)
#         y = np.flip(y, 1)
    return x

In [ ]:
train_dataset=LazyLoadDataset("/kaggle/input/lazydata2/lazydata/",transform = transform1)


In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=False)

In [ ]:
sample = train_dataset[0]

In [ ]:
(img0,img1,img2,depth,field_id),Y = sample

In [ ]:
plt.imshow(img0)

In [ ]:
Y

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import random
def train(epoch, model, optimizer, permute_pixels=None, permutation_order=None):
    """
    Train the model for one epoch

    Args:
        epoch (int): current epoch
        model (nn.Module): model to train
        optimizer (torch.optim): optimizer to use
        permute_pixels (function): function to permute the pixels (default: None)
        permutation_order (1D torch array): order of the permutation (default: None)
    """
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        
        randimg = random.randint(0,2)
        data2 = torch.stack((data[randimg][:,:,:,0],
                             data[randimg][:,:,:,1],
                             data[randimg][:,:,:,2],
                             data[3][:,randimg,:]),1)
        print(data2[0].shape)
        data2, target = data2.to(device), target.to(device)
        


        output = model(data2.float())
        mse_loss = nn.MSELoss()
        loss = mse_loss(output.float(), target.float())

        optimizer.zero_grad()#reset gradient to zeros before calculate new gradients
        loss.backward()#calculate gradients
        optimizer.step()#update weights
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data2), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
class CNN(nn.Module):
    def __init__(self, input_size, conv_feature, fc_feature, output_size):
        super(CNN, self).__init__()
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=conv_feature, kernel_size=65),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=conv_feature, out_channels=conv_feature, kernel_size=65), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )
        self.linear_layers=nn.Sequential(
            nn.Linear(conv_feature*8*8, fc_feature),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(fc_feature, output_size),
            
        )
        
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        finalresult = self.linear_layers(x)
        return finalresult
    

In [ ]:
# Training settings 
conv_features = 80 # number of features
fc_features = 50
output_size = 12
model_cnn = CNN(224*224,conv_features,fc_features,output_size) 
model_cnn.to(device)
optimizer = torch.optim.SGD(model_cnn.parameters(), lr=0.01, momentum=0.9) # use SGD with learning rate 0.01 and momentum 0.9


for epoch in range(0, 10):
    train(epoch, model_cnn, optimizer)

    


In [ ]:
torch.save(model_cnn.state_dict(), "model_cnn.pt")

In [ ]:

# model_ft = models.resnet50(pretrained=True)
# model_ft.conv1 = nn.Conv2d(4,64, 7, stride=2, padding=3, bias=False)
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 12)
# print(model_ft)

In [ ]:
# model_ft = model_ft.to(device)

In [ ]:

# optimizer = torch.optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9) # use SGD with learning rate 0.01 and momentum 0.9


# for epoch in range(0, 1):
#     train(epoch, model_ft, optimizer)


In [ ]:
# torch.save(model_ft.state_dict(), "model_ft.pt")

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
conv_features = 80 # number of feature maps
fc_features = 50
output_size = 12
model_cnn = CNN(224*224,conv_features,fc_features,output_size) # create CNN model
model_cnn.to(device)
model_cnn.load_state_dict(torch.load('/kaggle/working/model_cnn.pt'))
model_cnn.eval()

In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# model_ft.load_state_dict(torch.load('/kaggle/working/model_ft.pt'))
# model_ft.eval()

In [ ]:
test_data = torch.load('/kaggle/input/capstone-data/captstone_data/test/test/testX.pt')

file_ids = test_data[-1]


In [ ]:
test_1 = LazyLoadDataset("/kaggle/input/lazydata2/lazydata/",train = False, transform = transform1)

test_loader = torch.utils.data.DataLoader(test_1, batch_size=64, shuffle=False)

In [ ]:
len(test_loader)

In [ ]:
train_loader.dataset[3][1]

In [ ]:
preds2 = []



for batch_idx, data in enumerate(test_loader):
    
    randimg = random.randint(0,2)
    data3 = torch.stack((data[randimg][:,:,:,0],
                         data[randimg][:,:,:,1],
                         data[randimg][:,:,:,2],
                         data[3][:,randimg,:]),1)

    data3 = data3.float()
    print(data3[0].shape)
    output = model_cnn(data3.to(device))
    preds2.append(output.cpu().detach().numpy())

    

In [ ]:
train_loader.dataset[829][1]

In [ ]:
np.concatenate(preds2).shape


In [ ]:
import pandas as pd
np.concatenate(preds2).shape


np.concatenate(preds2)[843]


columnsT =  [ 'FINGER_POS_1', 'FINGER_POS_2', 'FINGER_POS_3', 'FINGER_POS_4', 'FINGER_POS_5', 'FINGER_POS_6',
         'FINGER_POS_7', 'FINGER_POS_8', 'FINGER_POS_9', 'FINGER_POS_10', 'FINGER_POS_11', 'FINGER_POS_12']
df = pd.DataFrame (np.concatenate(preds2), columns = columnsT)

df.insert(0, 'ID', file_ids)
outfile = 'submission.csv'

df.to_csv(outfile, index = False)
print("Written to csv file {}".format(outfile))
                   
# df


In [ ]:
df

In [ ]:
df.sample(20)